In [80]:
import csv

torah = 'torah.csv'
megillot = 'chamesh megillot.csv'
navi1 = 'neviim rishonim.csv'
navi2 = 'neviim achronim.csv'
emet = 'sifrei emet.csv'
ketuvim = 'ketuvim.csv'


cant = ['֑','֒','֓','֔','֕','֖','֗','֘','֙','֚','֛','֜',
             '֝','֞','֠','֡','֢','֣','֤','֥','֦','֧','֨','֩','֪','֫','֬','֭','֯','׃']

kamatz_katan = "ׇ"

meteg = "ֽ"

In [86]:
#removes trop but not vowels
def trop_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter not in cant and letter != meteg:
            no_cant_word = no_cant_word + letter
    return no_cant_word

In [130]:
def kamatz_katan_finder(bookset):
    words_with_katan = {"כׇּל":[],"כׇל":[]}
    questionable_words = []
    with open(bookset, 'r', encoding='utf-8') as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        in_range = False
        for row in csvReader:
            row[4] = row[4].replace("{{מ:לגרמיה}}"," {{מ:לגרמיה}} ")
            row[4] = row[4].replace("{{מ:פסק}}"," {{מ:פסק}} ")
            words = row[4].split(" ")
            for word in words:
                try:
                    if word[0:2] == "וּ" or word[0:2] == "וְ":
                        word = word[2:]
                    if word[1] == "ּ":
                        word = word[0]+word[2:]
                except:
                    word = word
                if "מ:קמץ" in word:
                    start = word.index("|ד=")+3
                    if "|ס=" in word:
                        end = word.index("|",start)
                    else:
                        end = len(word)-2
                    word = word[start:end]
                    if "־" in word:
                        divided = word.split("־")
                        for piece in divided:
                            if "ׇ" in piece:
                                questionable_words.append(piece)
                    else:
                        questionable_words.append(word)
                #elif "{{" in word and kamatz_katan in word:
                    
                if kamatz_katan not in word:
                    continue
                elif trop_remover(word) not in words_with_katan.keys() and "־" not in word:
                    words_with_katan[trop_remover(word)] = [row[3]]
                elif trop_remover(word) in words_with_katan.keys() and "־" not in word:
                    words_with_katan[trop_remover(word)].append(row[3])
                elif "־" in word:
                    divided = word.split("־")
                    for piece in divided:
                        piece = trop_remover(piece)
                        if kamatz_katan in piece and "כׇּל" not in piece and "כׇל" not in piece:
                            if trop_remover(piece) not in words_with_katan:
                                words_with_katan[piece] = [row[3]]
                            else:
                                words_with_katan[trop_remover(piece)].append(row[3])
    return words_with_katan, questionable_words

In [131]:
sefer_names = {
    "בראשית":"Gen",
    "שמות":"Ex",
    "ויקרא":"Lev",
    "במדבר":"Num",
    "דברים":"Deut"
}

letters_to_numbers = {
    "א":1,
    "ב":2,
    "ג":3,
    "ד":4,
    "ה":5,
    "ו":6,
    "ז":7,
    "ח":8,
    "ט":9,
    "י":10,
    "כ":20,
    "ל":30,
    "מ":40,
    "נ":50,
    "ס":60,
    "ע":70,
    "פ":80,
    "צ":90,
    "ק":100,
    "ר":200,
    "ש":300,
    "ת":400
}

In [132]:
def gematria_parser(string):
    result = 0
    for char in string:
        if char in letters_to_numbers.keys():
            result += letters_to_numbers[char]
    return result

In [133]:
def reference_parser(refstring):
    #{{מ:פסוק|בראשית|כ|יב}}
    book_start = refstring.index("פסוק|")+5
    book_end = refstring.index("|",book_start+1)-1
    book = refstring[book_start:book_end+1]
    chap_start = book_end+2
    chap_end = refstring.index("|",chap_start)
    chapter = refstring[chap_start:chap_end]
    verse_start = chap_end+1
    if "|" in refstring[verse_start:-1]:
        verse_end = refstring.index("|",verse_start+1)
    else:
        verse_end = refstring.index("}")
    verse = refstring[verse_start:verse_end]
    
    book_eng = sefer_names[book]
    chap_eng = gematria_parser(chapter)
    verse_eng = gematria_parser(verse)
    ref_eng = book_eng+" "+str(chap_eng)+":"+str(verse_eng)
    return ref_eng

In [134]:
 def exporter(katandict,questionable):
    with open("output.tex",'w',encoding='utf-8') as outfile:
        outfile.write("\documentclass[14pt]{article}\n")
        outfile.write("\\usepackage{kamatz_list_style}\n\\begin{document}\n")
        outfile.write("\\begin{itemize} ")
        for item in katandict:
            if "{" in item:
                continue
            outfile.write("\item \hebword{")
            outfile.write(item)
            outfile.write("}, occurs in ")
            for ref in katandict[item]:
                outfile.write(reference_parser(ref))
                if ref != katandict[item][-1]:
                    outfile.write(", ")
            if katandict[item] == []:
                outfile.write("many places")
            if item in questionable:
                outfile.write(". This kamatz katan is subject to debate between experts and reading traditions.")
            outfile.write("\n\n")
        outfile.write("\end{itemize}\n\end{document} ")

In [135]:
info = kamatz_katan_finder(torah)
exporter(info[0],info[1])